In [ ]:
#colab cell
#from google.colab import drive
#drive.mount('/content/drive')
#!unzip /content/drive/MyDrive/Colab\ Notebooks/Phase2.zip
!ls

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
# Load image file paths and corresponding noise labels from labels.xlsx
labels_df = pd.read_excel('labels.xlsx', names=['Original_Path', 'Noise_Type', 'Noisy_Path'])

target_size = 412

In [ ]:
# Preprocess the data
image_paths = labels_df['Original_Path'].tolist()
noise_labels = labels_df['Noise_Type'].tolist()


In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(image_paths, noise_labels, test_size=0.1, random_state=42)


In [ ]:
# Define a function to load and preprocess images
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (target_size, target_size))  # Resize the image to the desired input shape
    img = img / 255.0  # Normalize pixel values to the range [0, 1]
    return img


In [ ]:
# Load and preprocess images
X_train_processed = np.array([load_and_preprocess_image(path) for path in X_train])
X_val_processed = np.array([load_and_preprocess_image(path) for path in X_val])


In [ ]:
# Define the noise classifier model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(target_size, target_size, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # Output layer with 3 classes for the types of noise
])


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the noise classifier model
model.fit(X_train_processed, np.array(y_train), epochs=50, validation_data=(X_val_processed, np.array(y_val)))


In [ ]:
# Evaluate the noise classifier model
y_pred = model.predict(X_val_processed)
y_pred_classes = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_val, y_pred_classes)
conf_matrix = confusion_matrix(y_val, y_pred_classes)
classification_rep = classification_report(y_val, y_pred_classes)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)